Pablo Jimeno - UPV/EHU 2016

In [ ]:
%run 'Utilities_main.ipynb'

In [ ]:
## Import redMaPPer catalogue

cat_file = '%s/Original_redMaPPer_DR8/redmapper_dr8_public_v6.3_catalog.fits' % cat_dir

data = pyfits.open(cat_file)

cat_raw  = data[1].data

data.close()

cat = Table(cat_raw, copy=False)

In [ ]:
## Import redMaPPer angular mask

amask_file = cat_dir + '/Original_redMaPPer_DR8/redmapper_dr8_public_v6.3_zmask.fits'
amask_data = fits.getdata(amask_file)

NSIDE = 2048

amask_npixels = hp.nside2npix(NSIDE)

amask_final = np.zeros((amask_npixels,2),dtype=np.float32)

pixel_mask = amask_data['HPIX']-1

amask_final[:,0][pixel_mask] = amask_data['FRACGOOD']
amask_final[:,1][pixel_mask] = amask_data['ZMAX']

hp.mollview(amask_final[:,0], title="FRACGOOD", xsize=1000)
hp.mollview(amask_final[:,1], title="ZMAX", unit='z', min=0.55, max=0.6, xsize=1000)

In [ ]:
## Import SDSS catalogue

sdss_file = cat_dir + '/SDSS/specObj-dr12.fits'

boss = import_DR12(sdss_file, sample='BOSS')
sdss = import_DR12(sdss_file, sample='SDSS')

In [ ]:
print '\nlen cat: %i' % len(cat)

print '\ncat columns:\n%s' % cat.colnames

print '\nlen boss: %i' % len(boss['z'])
print '\nlen sdss: %i' % len(sdss['z'])

In [ ]:
### Create new SYNTHETIC redshift distributions:

# spec-z coming from SDSS or others of 1º CG:
cat_sdss = Table.Column(name='Z_SDSS', data=np.full((len(cat),5),-1.))
cat.add_column(cat_sdss, 10)

cat['Z_SDSS'][:,0] = cat['Z_SPEC']

cat_boss = Table.Column(name='Z_BOSS', data=np.full((len(cat),5),-1.))
cat.add_column(cat_boss, 11)

In [ ]:
print '\nOriginal catalog: number of most probable CGs with SDSS spec-z: %i ' % len(cat[cat['Z_SPEC']!=-1.])

sky_limit = 0.5 # Match: distance less than sky_limit arcsecs.

for CG in [0,1,2,3,4]:
    
    print '\nComputing matches for %iº most probable CGs...' % (CG+1)
    
    cat_sky = SkyCoord(ra = cat['RA_CEN'][:,CG], dec = cat['DEC_CEN'][:,CG], frame='icrs', unit='deg')

    for z_sample in ['Z_SDSS', 'Z_BOSS']:
        
        if z_sample=='Z_SDSS':
            gal=sdss
        elif z_sample=='Z_BOSS':
            gal=boss
            
        gal_sky = SkyCoord(ra = gal['ra'], dec = gal['dec'], frame='icrs', unit='deg')
        
        idx, d2d, d3d = cat_sky.match_to_catalog_sky(gal_sky)
        sky_dists = d2d.arcsecond
        sky_mask = sky_dists < sky_limit # Objects in redMaPPer with SDSS match.
        sky_matches = idx[sky_mask] # Objects in SDSS with redMaPPer match.
    
        cat[z_sample][:,CG][sky_mask] = gal['z'][sky_matches]    
        
        print 'Number of %iº most probable CGs with %s spec-z: %i' % ( CG+1, z_sample,len(cat[cat[z_sample][:,CG]!=-1.]))

In [ ]:
### Export completed redMaPPer catalog:

new_redm_file = cat_dir + '/redMaPPer_DR12_v6.3.fits'

if os.path.exists(new_redm_file):
    os.remove(new_redm_file)

cat.write(new_redm_file, format='fits')

In [ ]:
### Export completed redMaPPer catalog (text version):

raw_catalog = Table.read(new_redm_file, format='fits')

print '\nNumber of raw objects found: ', len(raw_catalog)
print '\nRaw columns:\n\n', raw_catalog.colnames

catalog = {}

catalog['id'] = raw_catalog['ID'].data
catalog['ra'] = raw_catalog['RA_CEN'].data[:,0]
catalog['dec'] = raw_catalog['DEC_CEN'].data[:,0]
catalog['z_spec'] = raw_catalog['Z_SDSS'].data[:,0]
catalog['z_photo'] = raw_catalog['Z_LAMBDA'].data
catalog['z_photo_err'] = raw_catalog['Z_LAMBDA_ERR'].data
catalog['rich'] = raw_catalog['LAMBDA'].data
catalog['rich_err'] = raw_catalog['LAMBDA_ERR'].data

catalog_out = np.array((catalog['id'], 
                        catalog['ra'],
                        catalog['dec'],
                        catalog['z_spec'],
                        catalog['z_photo'],
                        catalog['z_photo_err'],
                        catalog['rich'],
                        catalog['rich_err'])).T

# HEADER: ID / RA / DEC / SPEC_Z / PHOTO_Z / PHOTO_Z_ERR / RICHNESS / RICHNESS_ERR
np.savetxt(cat_dir + '/redMaPPer_DR12_v6.3.txt', catalog_out,
           fmt='%.i\t%.7f\t%.7f\t%.6f\t%.6f\t%.6f\t%.2f\t%.2f')